Hello
Below you will find a tutorial on how to use ATLAS_Parser - a tool made for working with ATLAS Open Data

We will start by importing and initializing configurations

In [ ]:
#Import the tools
from src.parse_atlas import parser
from src.calculations import physics_calcs
import math

In [ ]:
#Configurations
config = {
    "max_process_memory_mb": 8192,
    "max_chunk_size_bytes": 500_000_000,
    "max_threads": 3,
    "release_year": "2024",
    "file_limit": 5,
    "logging_path": "logs/",
}

config_kinematic_cuts = {
    "pt":{"min": 0},
    "eta":{"min": -5,"max": 5},
    "phi":{"min": -math.pi, "max": math.pi}
}

config_particle_counts = {
    "Electrons" : {"min":1, "max": 6},
    "Jets" : {"min":2, "max": 8},
    "Muons" : {"min":1, "max": 6},
    "Photons" : {"min":0, "max": 5}
}

Start by initilazing the ATLAS_Parser class and fetch record ids for records released in year 2024

In [ ]:
#Initializing the parser with configuration
atlasparser = parser.ATLAS_Parser(
        max_process_memory_mb=config["max_process_memory_mb"],
        max_chunk_size_bytes=config["max_chunk_size_bytes"],
        max_threads=config["max_threads"],
        logging_path=config["logging_path"]
        )

#Fetching file URIs for a specified release year
release_files_uris = atlasparser.fetch_records_ids(
    release_year=config["release_year"]
)

Now using these record ids we can parse using parse_files
Apply cuts and filters
And save the output

Using ATLAS_Parser class and physics_calcs

In [ ]:
for events_chunk in atlasparser.parse_files(
    files_ids=release_files_uris, 
    limit=config["file_limit"]
):
    #Here you can do stuff with events_chunk
    #Below you can find an example pipeline that processes the events
    
    #Cutting by kinematics
    cut_events = physics_calcs.filter_events_by_kinematics(
        events_chunk, config_kinematic_cuts
    ) 

    #Filtering events by amounts of each particle
    filtered_events = physics_calcs.filter_events_by_particle_counts(
        events=cut_events, 
        particle_counts=config_particle_counts, 
        is_particle_counts_range=True
    ) 

    #Prepare the events array for ROOT format
    root_ready = atlasparser.flatten_for_root(filtered_events)

    #Save the file
    atlasparser.save_events_as_root(root_ready, config["output_path"])